In [1]:
import json
import os
from typing import Optional, Tuple, List
from datetime import datetime
from pathlib import Path
from openai import OpenAI
import fire
import pandas as pd

import numpy as np
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.callbacks import BaseCallback

from alphagen.data.expression import *
from alphagen.data.parser import ExpressionParser
from alphagen.models.linear_alpha_pool import LinearAlphaPool, MseAlphaPool
from alphagen.rl.env.wrapper import AlphaEnv
from alphagen.rl.policy import LSTMSharedNet
from alphagen.utils import reseed_everything, get_logger
from alphagen.rl.env.core import AlphaEnvCore
from alphagen_qlib.calculator import QLibStockDataCalculator
from alphagen_qlib.stock_data import initialize_qlib
from alphagen_llm.client import ChatClient, OpenAIClient, ChatConfig
from alphagen_llm.prompts.system_prompt import EXPLAIN_WITH_TEXT_DESC
from alphagen_llm.prompts.interaction import InterativeSession, DefaultInteraction

In [1]:
import qlib
from qlib.data import D

qlib.init(provider_uri="~/.qlib/qlib_data/us_data_update", region="us")
df = D.features(D.instruments("sp500"), ["$open","$close","$high","$low","$vwap"], freq="day")
# df = D.features(["^GSPC"], ["$open","$close","$high","$low","$vwap"], freq="day")
df

[41028:MainThread](2025-09-02 09:02:26,384) INFO - qlib.Initialization - [config.py:420] - default_conf: client.
[41028:MainThread](2025-09-02 09:02:31,663) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[41028:MainThread](2025-09-02 09:02:31,664) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/tywat/.qlib/qlib_data/us_data_update')}


$open    $close     $high      $low     $vwap
instrument datetime                                                    
A          2020-11-10  1.023775  1.000000  1.024592  0.989020  1.037173
           2020-11-11  1.008530  0.992831  1.014428  0.987659  1.034228
           2020-11-12  0.992740  0.993103  1.007350  0.988203  1.032996
           2020-11-13  1.002359  0.999728  1.012160  0.996189  1.033480
           2020-11-16  1.007259  1.000635  1.011706  0.993376  1.033681
...                         ...       ...       ...       ...       ...
ZTS        2025-01-10  1.038854  1.037076  1.047744  1.034282  1.126193
           2025-01-13  1.038537  1.056126  1.059618  1.033012  1.126123
           2025-01-14  1.056761  1.043998  1.061333  1.026027  1.126003
           2025-01-15  1.054729  1.061523  1.066667  1.050220  1.125926
           2025-01-16  1.057015  1.075493  1.078097  1.053649  1.125867

[477360 rows x 5 columns]

In [3]:
import os
import shutil
import pandas as pd

# Path to your source CSVs
data_path = os.path.expanduser("~/.qlib/stock_data/source/us_data_update")
# backup_path = os.path.join(data_path, "backup")

# Make backup folder
# os.makedirs(backup_path, exist_ok=True)

for filename in os.listdir(data_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(data_path, filename)
        # backup_file = os.path.join(backup_path, filename)

        # Backup first
        # if not os.path.exists(backup_file):
        #     shutil.copy(file_path, backup_file)
        #     print(f"Backed up {filename} → {backup_file}")

        print(f"Processing {file_path}...")

        # Read CSV
        df = pd.read_csv(file_path)

        # Check required columns
        required_cols = {"high", "low", "close", "volume"}
        if not required_cols.issubset(df.columns):
            print(f"⚠️ Skipping {filename}, missing columns")
            continue

        # Compute VWAP (daily cumulative approximation)
        typical_price = (df["high"] + df["low"] + df["close"]) / 3
        df["vwap"] = (typical_price * df["volume"]).cumsum() / df["volume"].cumsum()
        df["vwap"] = df["vwap"].ffill().astype(float)

        # Save back (overwrite)
        df.to_csv(file_path, index=False)

print("✅ Finished adding VWAP to all CSV files. Original files are in 'backup' folder.")


Processing C:\Users\tywat/.qlib/stock_data/source/us_data_update\A.csv...
Processing C:\Users\tywat/.qlib/stock_data/source/us_data_update\AA.csv...
Processing C:\Users\tywat/.qlib/stock_data/source/us_data_update\AAA.csv...
Processing C:\Users\tywat/.qlib/stock_data/source/us_data_update\AAAU.csv...
Processing C:\Users\tywat/.qlib/stock_data/source/us_data_update\AACG.csv...
Processing C:\Users\tywat/.qlib/stock_data/source/us_data_update\AACT-UN.csv...
Processing C:\Users\tywat/.qlib/stock_data/source/us_data_update\AACT.csv...
Processing C:\Users\tywat/.qlib/stock_data/source/us_data_update\AADR.csv...
Processing C:\Users\tywat/.qlib/stock_data/source/us_data_update\AAL.csv...
Processing C:\Users\tywat/.qlib/stock_data/source/us_data_update\AAM-UN.csv...
Processing C:\Users\tywat/.qlib/stock_data/source/us_data_update\AAM.csv...
Processing C:\Users\tywat/.qlib/stock_data/source/us_data_update\AAME.csv...
Processing C:\Users\tywat/.qlib/stock_data/source/us_data_update\AAMI.csv...
Pr

In [2]:
instruments: str = "csi300"
device = torch.device("cuda:0")


def get_dataset(start: str, end: str) -> StockData:
    return StockData(
        instrument=instruments,
        start_time=start,
        end_time=end,
        device=device
    )

segments = [
    ("2012-01-01", "2019-12-31"),
    ("2022-01-01", "2022-06-30"),
    ("2022-07-01", "2022-12-31"),
    ("2023-01-01", "2023-06-30")
]


datasets = [get_dataset(*s) for s in segments]

[23512:MainThread](2025-04-24 12:33:13,832) INFO - qlib.Initialization - [config.py:420] - default_conf: client.
[23512:MainThread](2025-04-24 12:33:14,950) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[23512:MainThread](2025-04-24 12:33:14,952) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/tywat/.qlib/qlib_data/cn_data')}


In [3]:
close = Feature(FeatureType.CLOSE)
target = Ref(close, -20) / close - 1
calculators = [QLibStockDataCalculator(d, target) for d in datasets]

In [4]:
from alphagen.data.expression import Operators
from alphagen.data.parser import ExpressionParser

def load_linear_alpha_pool_from_json(json_path: str, 
                                     calculator: QLibStockDataCalculator,
                                     single_alpha: bool = False) -> LinearAlphaPool | list[LinearAlphaPool]:
    # Load the JSON file
    parser = ExpressionParser(Operators)
    with open(json_path, 'r') as f:
        pool_data = json.load(f)

    # Extract expressions and weights from the loaded data
    expressions = pool_data['exprs']
    weights = pool_data['weights']

    # Create an instance of LinearAlphaPool
    alpha_pool = MseAlphaPool(
        capacity=len(expressions),  # Set the capacity based on the number of expressions
        calculator=calculator
    )

    # Load the expressions into the pool
    expres = []
    if single_alpha:
        alpha_pools = []

        for expression,weight in zip(expressions,weights):
            alpha_pool = MseAlphaPool(
                capacity=1,
                calculator=calculator
                )
            expre = parser.parse(expression)
            alpha_pool.force_load_exprs([expre], [weight])
            alpha_pools.append(alpha_pool)

        return  alpha_pools
    else:
        for expression in expressions:
            expre = parser.parse(expression)
            expres.append(expre)
        
        
        alpha_pool.force_load_exprs(expres, weights)

        return alpha_pool

alpha_pools = load_linear_alpha_pool_from_json('out/results/csi300_20_0_20250208124320_rl/251904_steps_pool.json', calculators[1])
alpha_pool = load_linear_alpha_pool_from_json('out/results/csi300_20_0_20250208124320_rl/251904_steps_pool.json', calculators[1], single_alpha=True)

In [5]:
ic_value, rank_ic_value = alpha_pools.test_ensemble(calculators[2])
print(alpha_pools.exprs)
print(ic_value, rank_ic_value)

[Greater(Div(Div(-1.0,$high),EMA($open,10d)),-2.0), Delta(Log($vwap),1d), Mul($volume,Mul(Cov($close,Mul(5.0,Min(Mul($high,-30.0),40d)),40d),-0.01)), Sum(Mul(Corr(Div($vwap,-0.5),$close,5d),-10.0),10d), Abs(Sub(2.0,Div($close,Add(Greater(2.0,Delta(Log($low),5d)),30.0)))), Mad(Add(2.0,Mean($vwap,20d)),10d), Corr($close,$low,10d), Abs(Log(Mad(Sub(-0.5,$close),20d))), Mad(Log(Log($volume)),40d), Mul(0.5,Corr(Log($volume),WMA(Log($volume),40d),40d)), Mul(Mul($volume,Mul(Add(Mean($high,20d),30.0),$high)),0.5), Mul(WMA(Log(Abs(Var($low,5d))),20d),-2.0), Abs(Mul(5.0,Sub($open,30.0))), Mean(Less(Sub(-2.0,Corr($volume,$high,20d)),1.0),10d), Sub(Less(1.0,$low),5.0), Add(Corr(Sub(-1.0,$high),$volume,10d),0.01), WMA(Div(Std(WMA(Div(Div($vwap,30.0),$low),40d),20d),-5.0),10d), WMA(Sub(-1.0,Div($low,$close)),20d), Less(Div($close,$vwap),$volume), Sub(Mad(Mean(Log($low),20d),40d),5.0), None]
0.06614601612091064 0.0644562840461731


In [6]:
alpha_index = 3

ic_value, rank_ic_value = alpha_pool[alpha_index].test_ensemble(calculators[2])
print(alpha_pool[alpha_index].exprs)
print(ic_value, rank_ic_value)

[Sum(Mul(Corr(Div($vwap,-0.5),$close,5d),-10.0),10d), None]
0.010267447680234909 0.010892813093960285


In [7]:
for alpha in alpha_pool:
    print(alpha.exprs)

[Greater(Div(Div(-1.0,$high),EMA($open,10d)),-2.0), None]
[Delta(Log($vwap),1d), None]
[Mul($volume,Mul(Cov($close,Mul(5.0,Min(Mul($high,-30.0),40d)),40d),-0.01)), None]
[Sum(Mul(Corr(Div($vwap,-0.5),$close,5d),-10.0),10d), None]
[Abs(Sub(2.0,Div($close,Add(Greater(2.0,Delta(Log($low),5d)),30.0)))), None]
[Mad(Add(2.0,Mean($vwap,20d)),10d), None]
[Corr($close,$low,10d), None]
[Abs(Log(Mad(Sub(-0.5,$close),20d))), None]
[Mad(Log(Log($volume)),40d), None]
[Mul(0.5,Corr(Log($volume),WMA(Log($volume),40d),40d)), None]
[Mul(Mul($volume,Mul(Add(Mean($high,20d),30.0),$high)),0.5), None]
[Mul(WMA(Log(Abs(Var($low,5d))),20d),-2.0), None]
[Abs(Mul(5.0,Sub($open,30.0))), None]
[Mean(Less(Sub(-2.0,Corr($volume,$high,20d)),1.0),10d), None]
[Sub(Less(1.0,$low),5.0), None]
[Add(Corr(Sub(-1.0,$high),$volume,10d),0.01), None]
[WMA(Div(Std(WMA(Div(Div($vwap,30.0),$low),40d),20d),-5.0),10d), None]
[WMA(Sub(-1.0,Div($low,$close)),20d), None]
[Less(Div($close,$vwap),$volume), None]
[Sub(Mad(Mean(Log($low),

In [8]:
ics = []
rank_ics = []
alphas = []

for alpha in alpha_pool:
    ic_value, rank_ic_value = alpha.test_ensemble(calculators[2])

    ics.append(ic_value)
    rank_ics.append(rank_ic_value)
    alphas.append(alpha.exprs)

df_ic_ind = pd.DataFrame({'alpha': alphas, 'ic': ics, 'rank_ic': rank_ics})
df_ic_ind

,alpha,ic,rank_ic
0,"[Greater(Div(Div(-1.0,$high),EMA($open,10d)),-...",0.055708,0.084874
1,"[Delta(Log($vwap),1d), None]",-0.024560,-0.012933
2,"[Mul($volume,Mul(Cov($close,Mul(5.0,Min(Mul($h...",-0.036502,-0.035545
3,"[Sum(Mul(Corr(Div($vwap,-0.5),$close,5d),-10.0...",0.010267,0.010893
4,"[Abs(Sub(2.0,Div($close,Add(Greater(2.0,Delta(...",-0.061593,-0.092103
5,"[Mad(Add(2.0,Mean($vwap,20d)),10d), None]",-0.006636,0.043511
6,"[Corr($close,$low,10d), None]",0.056122,0.063845
7,"[Abs(Log(Mad(Sub(-0.5,$close),20d))), None]",-0.081099,-0.097603
8,"[Mad(Log(Log($volume)),40d), None]",-0.025405,-0.042726
9,"[Mul(0.5,Corr(Log($volume),WMA(Log($volume),40...",-0.040783,-0.046877


In [9]:
for p in Path("out/gp").iterdir():
    seed = int(p.name)

with open(p / "40.json") as f:
    report = json.load(f)


state = report["res"]["res"]["pool_state"]
state["exprs"]

['Std(EMA(Min(Mul(5.0,$high),30d),50d),10d)',
 'Std(EMA(Min(Log($vwap),30d),40d),10d)',
 'Sum(Mean(Abs(Corr($low,$high,20d)),40d),20d)',
 'Std(Med(Min(Mul(5.0,$high),30d),10d),10d)',
 'Mad(Min($low,20d),20d)',
 'Std(Std(Min(Mul($vwap,2.0),30d),10d),10d)',
 'Mad(Med($close,50d),10d)',
 'Std(Cov(Corr(Var($volume,40d),$high,20d),$close,30d),10d)',
 'Std(Med(Ref(Mul(10.0,$high),30d),10d),10d)',
 'Std(Min(Sum(Mul(10.0,$high),40d),50d),10d)',
 'Mad(Min($high,30d),10d)',
 'Std(Std(Med(Mul(0.5,$high),20d),20d),10d)',
 'Mad(Ref(Min($high,30d),10d),10d)',
 'Std(Abs(WMA(Cov(0.01,$high,50d),10d)),10d)',
 'Std(Min(WMA($high,20d),50d),10d)',
 'Log(Var(Sum($low,30d),40d))',
 'Std(EMA(Min(Mul(Std($high,10d),$high),30d),50d),10d)',
 'Std(Max(Min(Mul(5.0,$high),30d),20d),10d)',
 'Std(Min(Mean(Corr(5.0,$high,30d),50d),10d),10d)',
 'Std(EMA(WMA($vwap,10d),40d),10d)']

# main

In [19]:
ex_num = "us"

In [20]:
import pickle

file_path = f'out/backtests/{ex_num}/gp/2-graph.pkl'

with open(file_path, 'rb') as file:
    chart = pickle.load(file)
chart.show()

In [21]:
import pickle

file_path = f'out/backtests/{ex_num}/gp/2-report.pkl'

with open(file_path, 'rb') as file:
    gp_report = pickle.load(file)
gp_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench
datetime,,,,,,,,,
2022-01-31,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,0.018886
2022-02-01,9.985751e+07,-7.566996e-17,9.499596e+07,0.949960,1.424939e+05,0.001425,9.499596e+07,4.861546e+06,0.006863
2022-02-02,1.009831e+08,1.151148e-02,1.109366e+08,0.159634,1.664048e+05,0.000239,1.004648e+08,5.183356e+05,0.009423
2022-02-03,9.880965e+07,-2.123738e-02,1.301623e+08,0.190385,1.952434e+05,0.000286,9.831782e+07,4.918271e+05,-0.024391
2022-02-04,9.895345e+07,1.781089e-03,1.516222e+08,0.217184,2.274333e+05,0.000326,9.840696e+07,5.464832e+05,0.005157
...,...,...,...,...,...,...,...,...,...
2023-12-22,8.962899e+07,2.277941e-03,7.681024e+09,0.151741,1.152154e+07,0.000228,8.928380e+07,3.451903e+05,0.001660
2023-12-26,9.023729e+07,7.019237e-03,7.694906e+09,0.154888,1.154236e+07,0.000232,8.988305e+07,3.542364e+05,0.004232
2023-12-27,9.020569e+07,-1.716084e-04,7.705647e+09,0.119025,1.155847e+07,0.000179,8.992953e+07,2.761659e+05,0.001431


In [22]:
import pickle

file_path = f'out/backtests/{ex_num}/rl/0-graph.pkl'

with open(file_path, 'rb') as file:
    chart = pickle.load(file)
chart.show()

In [23]:
import pickle

file_path = f'out/backtests/{ex_num}/rl/0-report.pkl'

with open(file_path, 'rb') as file:
    alphaGen_report = pickle.load(file)
alphaGen_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench
datetime,,,,,,,,,
2022-01-31,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,0.018886
2022-02-01,9.985750e+07,2.022716e-16,9.499742e+07,0.949974,1.424961e+05,0.001425,9.499742e+07,4.860086e+06,0.006863
2022-02-02,9.919072e+07,-6.390652e-03,1.140853e+08,0.191151,1.711280e+05,0.000287,9.859373e+07,5.969897e+05,0.009423
2022-02-03,9.673376e+07,-2.448627e-02,1.328493e+08,0.189171,1.992740e+05,0.000284,9.625140e+07,4.823574e+05,-0.024391
2022-02-04,9.697439e+07,2.783992e-03,1.519637e+08,0.197598,2.279456e+05,0.000296,9.648650e+07,4.878968e+05,0.005157
...,...,...,...,...,...,...,...,...,...
2023-12-22,8.760690e+07,2.245135e-03,8.555176e+09,0.195773,1.283276e+07,0.000294,8.716982e+07,4.370802e+05,0.001660
2023-12-26,8.802381e+07,5.065349e-03,8.573080e+09,0.204375,1.285962e+07,0.000307,8.756712e+07,4.566857e+05,0.004232
2023-12-27,8.789446e+07,-1.177312e-03,8.590221e+09,0.194732,1.288533e+07,0.000292,8.745618e+07,4.382837e+05,0.001431


In [24]:
alphaGen_report["cum_return"] = alphaGen_report["return"].cumsum()
alphaGen_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench,cum_return
datetime,,,,,,,,,,
2022-01-31,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,0.018886,0.000000e+00
2022-02-01,9.985750e+07,2.022716e-16,9.499742e+07,0.949974,1.424961e+05,0.001425,9.499742e+07,4.860086e+06,0.006863,2.022716e-16
2022-02-02,9.919072e+07,-6.390652e-03,1.140853e+08,0.191151,1.711280e+05,0.000287,9.859373e+07,5.969897e+05,0.009423,-6.390652e-03
2022-02-03,9.673376e+07,-2.448627e-02,1.328493e+08,0.189171,1.992740e+05,0.000284,9.625140e+07,4.823574e+05,-0.024391,-3.087692e-02
2022-02-04,9.697439e+07,2.783992e-03,1.519637e+08,0.197598,2.279456e+05,0.000296,9.648650e+07,4.878968e+05,0.005157,-2.809293e-02
...,...,...,...,...,...,...,...,...,...,...
2023-12-22,8.760690e+07,2.245135e-03,8.555176e+09,0.195773,1.283276e+07,0.000294,8.716982e+07,4.370802e+05,0.001660,6.293587e-02
2023-12-26,8.802381e+07,5.065349e-03,8.573080e+09,0.204375,1.285962e+07,0.000307,8.756712e+07,4.566857e+05,0.004232,6.800122e-02
2023-12-27,8.789446e+07,-1.177312e-03,8.590221e+09,0.194732,1.288533e+07,0.000292,8.745618e+07,4.382837e+05,0.001431,6.682391e-02


In [25]:
import pickle

file_path = f'out/backtests/{ex_num}/boot/0-graph.pkl'

with open(file_path, 'rb') as file:
    chart = pickle.load(file)
chart.show()

In [26]:
import pickle

file_path = f'out/backtests/{ex_num}/boot/0-report.pkl'

with open(file_path, 'rb') as file:
    boot_report = pickle.load(file)
boot_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench
datetime,,,,,,,,,
2022-01-31,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,0.018886
2022-02-01,9.985750e+07,1.426088e-16,9.499807e+07,0.949981,1.424971e+05,0.001425,9.499807e+07,4.859436e+06,0.006863
2022-02-02,9.909104e+07,-7.328803e-03,1.180799e+08,0.231148,1.771198e+05,0.000347,9.839503e+07,6.960150e+05,0.009423
2022-02-03,9.688108e+07,-2.201536e-02,1.370404e+08,0.191344,2.055606e+05,0.000287,9.639092e+07,4.901556e+05,-0.024391
2022-02-04,9.646441e+07,-4.010919e-03,1.557616e+08,0.193240,2.336425e+05,0.000290,9.598600e+07,4.784106e+05,0.005157
...,...,...,...,...,...,...,...,...,...
2023-12-22,8.357639e+07,1.887509e-03,8.240504e+09,0.205581,1.236076e+07,0.000308,8.313869e+07,4.377048e+05,0.001660
2023-12-26,8.399096e+07,5.261658e-03,8.257292e+09,0.200859,1.238594e+07,0.000301,8.356200e+07,4.289563e+05,0.004232
2023-12-27,8.394164e+07,-2.898379e-04,8.273941e+09,0.198230,1.241091e+07,0.000297,8.351636e+07,4.252783e+05,0.001431


In [27]:
import pickle

file_path = f'out/backtests/{ex_num}/mcts/0-graph.pkl'

with open(file_path, 'rb') as file:
    chart = pickle.load(file)
chart.show()

In [28]:
import pickle

file_path = f'out/backtests/{ex_num}/mcts/0-report.pkl'

with open(file_path, 'rb') as file:
    riskminer_report = pickle.load(file)
riskminer_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench
datetime,,,,,,,,,
2022-01-31,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,0.018886
2022-02-01,9.985751e+07,3.894093e-16,9.499345e+07,0.949934,1.424902e+05,0.001425,9.499345e+07,4.864065e+06,0.006863
2022-02-02,1.007797e+08,9.586281e-03,1.183554e+08,0.233953,1.775331e+05,0.000351,1.000766e+08,7.031386e+05,0.009423
2022-02-03,9.929614e+07,-1.443962e-02,1.372704e+08,0.187687,2.059057e+05,0.000282,9.880753e+07,4.886106e+05,-0.024391
2022-02-04,9.863279e+07,-6.373401e-03,1.575986e+08,0.204723,2.363980e+05,0.000307,9.811409e+07,5.187042e+05,0.005157
...,...,...,...,...,...,...,...,...,...
2023-12-22,9.263615e+07,1.921813e-03,8.614892e+09,0.200700,1.292234e+07,0.000301,9.216154e+07,4.746054e+05,0.001660
2023-12-26,9.298522e+07,4.064197e-03,8.633172e+09,0.197326,1.294976e+07,0.000296,9.251814e+07,4.670757e+05,0.004232
2023-12-27,9.299568e+07,4.115792e-04,8.651711e+09,0.199382,1.297757e+07,0.000299,9.252244e+07,4.732445e+05,0.001431


In [29]:
import pickle

file_path = f'out/backtests/{ex_num}/emcts/0-report.pkl'

with open(file_path, 'rb') as file:
    eminer_report = pickle.load(file)
eminer_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench
datetime,,,,,,,,,
2022-01-31,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,0.018886
2022-02-01,9.985750e+07,-1.656008e-16,9.499677e+07,0.949968,1.424951e+05,0.001425,9.499677e+07,4.860739e+06,0.006863
2022-02-02,9.919604e+07,-6.279433e-03,1.179435e+08,0.229795,1.769152e+05,0.000345,9.850350e+07,6.925398e+05,0.009423
2022-02-03,9.691736e+07,-2.261181e-02,1.417287e+08,0.239780,2.125931e+05,0.000360,9.630793e+07,6.094239e+05,-0.024391
2022-02-04,9.746656e+07,5.992189e-03,1.627550e+08,0.216951,2.441325e+05,0.000325,9.692773e+07,5.388386e+05,0.005157
...,...,...,...,...,...,...,...,...,...
2023-12-22,9.763808e+07,1.517543e-03,8.839968e+09,0.200887,1.325995e+07,0.000301,9.713732e+07,5.007565e+05,0.001660
2023-12-26,9.814968e+07,5.540936e-03,8.859570e+09,0.200764,1.328936e+07,0.000301,9.764911e+07,5.005729e+05,0.004232
2023-12-27,9.813202e+07,1.075770e-04,8.878384e+09,0.191680,1.331758e+07,0.000288,9.765081e+07,4.812090e+05,0.001431


In [30]:
import pickle

file_path = f'out/backtests/{ex_num}/oracle/0-report.pkl'

with open(file_path, 'rb') as file:
    oracle_report = pickle.load(file)
oracle_report

,account,return,total_turnover,turnover,total_cost,cost,value,cash,bench
datetime,,,,,,,,,
2022-01-31,1.000000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,1.000000e+08,0.018886
2022-02-01,9.985750e+07,3.929017e-17,9.499759e+07,0.949976,1.424964e+05,0.001425,9.499759e+07,4.859915e+06,0.006863
2022-02-02,9.953163e+07,-2.921219e-03,1.177796e+08,0.228145,1.766693e+05,0.000342,9.884300e+07,6.886211e+05,0.009423
2022-02-03,9.652325e+07,-2.993687e-02,1.369199e+08,0.192304,2.053799e+05,0.000288,9.602929e+07,4.939566e+05,-0.024391
2022-02-04,9.649657e+07,5.223379e-06,1.550399e+08,0.187727,2.325599e+05,0.000282,9.603273e+07,4.638420e+05,0.005157
...,...,...,...,...,...,...,...,...,...
2023-12-21,8.628698e+07,1.906901e-02,7.864353e+09,0.204221,1.179653e+07,0.000306,8.584537e+07,4.416130e+05,0.010301
2023-12-22,8.642150e+07,1.851527e-03,7.881185e+09,0.195072,1.182178e+07,0.000293,8.599122e+07,4.302785e+05,0.001660
2023-12-26,8.694761e+07,6.396001e-03,7.898947e+09,0.205523,1.184842e+07,0.000308,8.649473e+07,4.528769e+05,0.004232


In [31]:
import pandas as pd
df_com = pd.DataFrame()

df_com["GP"] = gp_report["return"].cumsum()
df_com["Alpha Gen"] = alphaGen_report["return"].cumsum()
df_com["Bootstrapped DQN"] = boot_report["return"].cumsum()
df_com["Oracle"] = oracle_report["return"].cumsum()
df_com["MCTS"] = riskminer_report["return"].cumsum()
df_com["EMCTS"] = eminer_report["return"].cumsum()
df_com["Benchmark"] = boot_report["bench"].cumsum()


df_com.head()

,GP,Alpha Gen,Bootstrapped DQN,Oracle,MCTS,EMCTS,Benchmark
datetime,,,,,,,
2022-01-31,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.018886
2022-02-01,-7.566996e-17,2.022716e-16,1.426088e-16,3.929017e-17,3.894093e-16,-1.656008e-16,0.025749
2022-02-02,1.151148e-02,-6.390652e-03,-7.328803e-03,-2.921219e-03,9.586281e-03,-6.279433e-03,0.035172
2022-02-03,-9.725900e-03,-3.087692e-02,-2.934416e-02,-3.285809e-02,-4.853336e-03,-2.889124e-02,0.010780
2022-02-04,-7.944811e-03,-2.809293e-02,-3.335508e-02,-3.285287e-02,-1.122674e-02,-2.289905e-02,0.015937


In [32]:
import pickle

rmse_files = []
df_rmse = pd.DataFrame()

for model in ["boot","gp","rl","mcts","emcts"]:
    file_path = f'out/backtests/{ex_num}/{model}/0-rmse.pkl'

    if model == "gp":
        file_path = f'out/backtests/{ex_num}/{model}/2-rmse.pkl'

    with open(file_path, 'rb') as file:
        rmse = pickle.load(file)
    
    df_rmse[model] = rmse["rmse"]

df_rmse.rename(columns={"boot": "Bootstrapped DQN", "gp": "GP", "rl": "Alpha Gen", "mcts":"RiskMiner", "emcts":"EMCTS"}, inplace=True)
df_rmse

,Bootstrapped DQN,GP,Alpha Gen,RiskMiner,EMCTS
date,,,,,
2022-01-31,154.335352,193.968746,148.339814,160.736343,162.430480
2022-02-01,178.205482,190.457454,189.838617,186.791905,187.061252
2022-02-02,193.227913,198.542464,216.577351,196.048418,213.441065
2022-02-03,189.105779,193.408016,201.039216,188.979674,203.782762
2022-02-04,166.861935,191.811805,172.038740,192.461021,171.884252
...,...,...,...,...,...
2023-12-22,170.533279,196.831140,173.138138,175.582479,174.663578
2023-12-26,173.062140,199.966748,196.665568,192.345819,181.558264
2023-12-27,176.926140,198.041651,185.557666,180.928211,195.395172


In [33]:
df_rmse_ma = df_rmse.rolling(30).mean()

In [34]:
df_com.columns

Index(['GP', 'Alpha Gen', 'Bootstrapped DQN', 'Oracle', 'MCTS', 'EMCTS',
       'Benchmark'],
      dtype='object')

In [35]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1,subplot_titles=["Cumulative Return", "RMSE"])


for col in ['GP', 'Alpha Gen', 'Bootstrapped DQN', 'MCTS', 'EMCTS', 'Benchmark']:#df_com.columns:
    fig.add_trace(
        go.Scatter(
            x=df_com.index,   
            y=df_com[col],    
            mode='lines',
            name=col,
            legendgroup = '1',
        ),
        row=1, 
        col=1
    )

for col in ['GP', 'Alpha Gen', 'Bootstrapped DQN', 'RiskMiner', 'EMCTS']:#df_rmse_ma.columns:
    fig.add_trace(
        go.Scatter(
            x=df_rmse_ma.index,
            y=df_rmse_ma[col],    
            mode='lines',
            name=col,
            legendgroup = '2',
        ),
        row=2, 
        col=1
    )

# Update the layout to add the title and template
fig.update_layout(
    template='seaborn',
    autosize=False,
    width=1200,
    height=1200,
    legend_tracegroupgap=580,
    legend_groupclick="toggleitem"
)

fig.show()


# fig = px.line(df_com, y=["Bootstrapped DQN","Alpha Gen","GP","Benchmark"], 
#             #   x="lifeExp", 
#               title='Cumulative Return',
#               template="seaborn",
#               )
# fig.show()

# Test

In [72]:
from alphagen_qlib.stock_data import StockData

data = StockData(
        instrument="csi300",
        start_time="2020-01-01",
        end_time="2022-01-01"
    )
data

In [ ]:
from qlib.data import D

instruments = data.stock_ids.tolist()

# Determine the proper start and end times for fetching price data.
# Here we use the same dates as in your StockData instance.
start_time = data._dates[data.max_backtrack_days].strftime("%Y-%m-%d")
end_time = data._dates[-data.max_future_days - 1].strftime("%Y-%m-%d")

# Query Qlib to get the closing price for each instrument.
# The field '$close' is used here (adjust if your field naming is different)
price_df = D.features(
    instruments=instruments,
    fields=["$close"],
    start_time="2020-01-01",
    end_time="2022-01-01"
)

price_df = price_df.reorder_levels(order=[1, 0])
price_df

In [ ]:
def compute_oracle_scores(price_df: pd.DataFrame) -> pd.DataFrame:
    # price_df is expected to be a MultiIndex DataFrame with (date, instrument)
    # Unstack to get dates as rows and instruments as columns
    price_unstacked = price_df.unstack(level=1)
    # Compute daily percentage returns and shift so that prediction on day t 
    # is compared with return from t to t+1
    oracle_signal = price_unstacked.pct_change().shift(-1)
    # Stack back to a MultiIndex DataFrame
    return oracle_signal.stack()


oracle_scores = compute_oracle_scores(price_df)
oracle_scores

C:\Users\tywat\AppData\Local\Temp\ipykernel_29000\2809321555.py:7: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

C:\Users\tywat\AppData\Local\Temp\ipykernel_29000\2809321555.py:9: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



$close
datetime   instrument          
2020-01-02 SH600000    0.010406
           SH600004   -0.007961
           SH600009   -0.000765
           SH600010    0.008123
           SH600011    0.000000
...                         ...
2021-12-30 SZ300782   -0.024249
           SZ300866   -0.033949
           SZ300888    0.008192
           SZ300896   -0.012722
           SZ300999    0.000476

[197057 rows x 1 columns]

In [17]:
from alphagen_qlib.utils import load_alpha_pool_by_path

calc = QLibStockDataCalculator(data, None)

for p in Path("out/boot_dqn").iterdir():
        inst, size, seed, time, ver = p.name.split('_', 4)
        size, seed = int(size), int(seed)
        if inst != "csi300" or size != 20 or time < "20240923" or ver == "llm_d5":
            continue
        try:
            exprs, weights = load_alpha_pool_by_path(str(p / "249500_steps_pool.json"))
        except:
            continue

boot_score = data.make_dataframe(calc.make_ensemble_alpha(exprs, weights))
boot_score

0
datetime   instrument          
2020-01-02 SH600000    0.029499
           SH600004    0.065036
           SH600009   -0.070193
           SH600010    0.009689
           SH600011    0.194273
...                         ...
2021-12-31 SZ300782   -0.121243
           SZ300866   -0.022676
           SZ300888    0.029904
           SZ300896   -0.026693
           SZ300999    0.017917

[200718 rows x 1 columns]

In [18]:
def normalize_series(series: pd.Series) -> pd.Series:
    return (series - series.mean()) / series.std()

def rank_series_per_date(series: pd.Series) -> pd.Series:
    """
    Rank the series for each date (assumed to be the first level of the MultiIndex).
    The highest value is assigned rank 1.
    """
    return series.groupby(level=0).rank(ascending=False, method='min')
def compute_rmse_per_date(model_scores: pd.Series, oracle_scores: pd.Series) -> pd.DataFrame:
    """
    Compute the RMSE across stocks for each date.
    
    Parameters:
      model_scores: pd.Series with MultiIndex (date, instrument) containing your model's prediction scores.
      oracle_scores: pd.Series with MultiIndex (date, instrument) containing the oracle's prediction scores.
      
    Returns:
      A DataFrame with the date as the index and a column 'rmse' containing the RMSE for that date.
    """
    # normalize
    # model_scores = normalize_series(model_scores)
    # oracle_scores = normalize_series(oracle_scores)

    # rank the scores
    model_scores = rank_series_per_date(model_scores)
    oracle_scores = rank_series_per_date(oracle_scores)

    # Combine both series into one DataFrame
    df = pd.DataFrame({
        "model": model_scores,
        "oracle": oracle_scores
    })
    # Group by the date level. If your MultiIndex doesn't have names,
    # you can group by level=0 (assuming the first level is the date).
    rmse_series = df.groupby(level=0).apply(
        lambda group: np.sqrt(((group["oracle"] - group["model"]) ** 2).mean())
    )
    rmse_df = rmse_series.to_frame(name="rmse")
    # Ensure the index is named "date" (or adjust as needed)
    rmse_df.index.name = "date"
    return rmse_df

rmse_df = compute_rmse_per_date(boot_score.iloc[:,0], oracle_scores.iloc[:,0])
rmse_df

,rmse
date,
2020-01-02,175.373923
2020-01-03,172.032975
2020-01-06,158.534962
2020-01-07,143.087359
2020-01-08,158.674098
...,...
2021-12-27,165.069690
2021-12-28,162.478913
2021-12-29,165.498930


In [19]:
import plotly.express as px
fig = px.line(rmse_df, y=["rmse"], 
            #   x="lifeExp", 
              title='RMSE',
              template="seaborn",
              )
fig.show()